In [ ]:
import xarray as xr
import ultraplot as uplt
import numpy as np

# Cargar archivos de validación (Hindcast)
try:
    cnn = xr.open_dataset('../outputs/hindcast_cnn.nc')['prw']
    mlp = xr.open_dataset('../outputs/hindcast_mlp.nc')['prw']
except:
    print("❌ Faltan archivos hindcast_*.nc")

# Configurar estilo PRO
uplt.rc.update(linewidth=0.8, ticklabelsize=10, titleloc='c')

# Seleccionar un mes intermedio para visualizar (ej: mes 6)
t_idx = 6
time_str = str(cnn.time.values[t_idx])[:7]

# Diseño: 2 filas (Mapas Reales, Mapas Diferencia)
fig, axs = uplt.subplots(nrows=2, ncols=2, proj='robin', width='12in', height='8in')

axs.format(
    coast=True, borders=True, 
    suptitle=f'Validación de Arquitecturas: {time_str} (Hindcast)',
    abc=True, abcloc='ul'
)

# --- FILA 1: PREDICCIONES ABSOLUTAS ---
# Usamos 'Spectral_r' que es muy colorido y científico (Rojo=Seco, Azul=Húmedo)
cmap_main = 'Spectral_r' 
levels = np.linspace(0, 70, 25)

# MLP
m1 = axs[0].contourf(mlp.isel(time=t_idx), cmap=cmap_main, levels=levels)
axs[0].set_title('Predicción MLP (Fully Connected)')

# CNN
m2 = axs[1].contourf(cnn.isel(time=t_idx), cmap=cmap_main, levels=levels)
axs[1].set_title('Predicción CNN (Convolutional)')

fig.colorbar(m1, loc='r', label='Vapor de Agua (kg/m²)', rows=1)

# --- FILA 2: DIFERENCIA ENTRE MODELOS ---
# ¿En qué difieren? (CNN - MLP)
diff = cnn.isel(time=t_idx) - mlp.isel(time=t_idx)

# Usamos mapa divergente 'Balance' (Azul=CNN más húmedo, Rojo=MLP más húmedo)
cmap_diff = 'Balance'
levels_diff = np.linspace(-15, 15, 21)

# Plot en ambos ejes de abajo (fusionados visualmente)
m3 = axs[2].contourf(diff, cmap=cmap_diff, levels=levels_diff, extend='both')
axs[2].set_title('Divergencia Espacial (CNN - MLP)')

# Histograma de errores (Extra pro)
ax_hist = axs[3]
data_diff = diff.values.flatten()
ax_hist.hist(data_diff, bins=50, color='gray7')
ax_hist.format(title='Distribución de Diferencias', xlabel='Diferencia (kg/m²)', ylabel='Frecuencia')
ax_hist.axvline(0, color='k', lw=1, ls='--')

fig.colorbar(m3, loc='b', label='Diferencia (kg/m²)', cols=(1,2))

# Guardar
fig.save('../outputs/figures/hindcast_professional_comparison.png', dpi=300)
plt.show()

/tmp/ipykernel_3400600/693938074.py:13: UltraPlotWarning: The rc setting 'linewidth' was deprecated in version 0.8.0 and may be removed in the next major release (version 2.0.0). Please use 'meta.width' instead.
  uplt.rc.update(linewidth=0.8, ticklabelsize=10, titleloc='c')
/tmp/ipykernel_3400600/693938074.py:20: UltraPlotWarning: GeoAxes can only be shared for rectilinear projections, ax.projection=<Projected CRS: +proj=robin +a=6378137.0 +lon_0=0 +no_defs +type=c ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Robinson
Datum: unknown
- Ellipsoid: unknown
- Prime Meridian: Greenwich
 is not a rectilinear projection.
  fig, axs = uplt.subplots(nrows=2, ncols=2, proj='robin', width='12in', height='8in')
